### 用于数组的文件输入输出
#### 将数组以二进制格式保存在磁盘
np.save和np.load是读写磁盘数组数据的两个主要函数。默认情况下，数组是以未压缩的原始二进制格式保存在扩展名为.npy的文件中的。

In [1]:
import numpy as np

In [2]:
arr = np.arange(10)

In [3]:
np.save('some_array',arr)

如果文件路径末尾没有扩展名.npy，则该扩展名会被自动加上。然后就可以通过np.load读取磁盘上的数组

In [4]:
np.load('some_array.npy')

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

可以使用np.savez将多个数组保存到一个压缩文件中，将数组以关键字参数的形式传入即可

In [5]:
np.savez('array_archive', a=arr, b=arr)

加载.npz文件时，会得到一个类似字典的对象，该对象会对各个数组进行延迟加载

In [6]:
arch = np.load('array_archive.npz')

In [7]:
arch['a']

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

#### 将数组读写到文本文件中
Numpy中用于加载文本文件的函数为numpy.loadtxt和numpy.genfromtxt。这些函数有很多的选项可供使用：指定各种分隔符、针对特定列的转换器函数、需要跳过的行数等，下面以简单的逗号分隔符为例 ：

In [8]:
!type array.txt

-2.532492620480090162e-01,2.765936601852464483e-01,5.380714669488444457e-01,-7.896639131809790602e-01,-3.222210887187342693e-02,-1.018003586610838562e+00
1.213488405550275928e+00,-6.220728888867192996e-01,-6.529117967204182138e-01,-5.447306090518424515e-01,-8.911875794121788541e-01,1.737147807966097135e+00
-1.953944883268158572e-01,1.387108324301131457e-01,2.693497040414932209e-01,-5.084479242696977597e-01,9.037152246714136306e-01,1.323178837794458385e-01
-1.136228364766184717e-01,6.799519436471701539e-01,1.026588254909145181e+00,1.145397231787889242e-01,-2.023638758934491300e+00,-4.579332265034087857e-01


In [9]:
np.loadtxt('array.txt', delimiter=',')

array([[-0.25324926,  0.27659366,  0.53807147, -0.78966391, -0.03222211,
        -1.01800359],
       [ 1.21348841, -0.62207289, -0.6529118 , -0.54473061, -0.89118758,
         1.73714781],
       [-0.19539449,  0.13871083,  0.2693497 , -0.50844792,  0.90371522,
         0.13231788],
       [-0.11362284,  0.67995194,  1.02658825,  0.11453972, -2.02363876,
        -0.45793323]])

numpy.savetxt执行的相反的操作：将数组数据写到以某种分隔符隔开的文本文件中，

In [11]:
np.savetxt('array.txt',arr,delimiter=',')

genfromtxt跟loadtxt差不多，只不过它面向的是结构化数组和缺失数据处理

#### 结构化和记录式数组
ndarray是一种同质数据容器，也就是说它表示的内存块中，各元素占用的字节数相同（具体根据dtype而定）。结构化数组是一种特殊的ndarray，其中的各个元素可以被看做C语言的结构体(struct，这也是“结构化”的由来)或SQL表中带有多个字段的行：

定义结构化dtype的方法有很多，最典型的方法是元组列表，各元组的格式为(field_name, field_data_type)。这样数组的元素就成了元组式的对象，该对象中各个元素可以像字典那样进行访问：

In [30]:
dtype = [('x', np.float64), ('y', np.int32)]

In [37]:
arr = np.array([(1.5,2.3), (np.pi, -2)], dtype = dtype)

In [38]:
arr

array([(1.5       ,  2), (3.14159265, -2)],
      dtype=[('x', '<f8'), ('y', '<i4')])

In [18]:
arr[0]

(1.5, 2)

In [19]:
arr[0]['y']

2

字段名保存在dtype.names属性中。在访问结构化数组的某个字段时，返回的是该数据的视图，所以不会发生数据复制:

In [20]:
arr['y']

array([ 2, -2])

#### 嵌套dtype和多维字段
在定义结构化dtype时，可以再设置一个形状（可以是一个整数，也可以是一个元组）：

In [25]:
dtype = [('x', np.float64, 3), ('y', np.int32)]

In [26]:
arr = np.zeros(4, dtype = dtype)

In [27]:
arr

array([([0., 0., 0.], 0), ([0., 0., 0.], 0), ([0., 0., 0.], 0),
       ([0., 0., 0.], 0)], dtype=[('x', '<f8', (3,)), ('y', '<i4')])

在这个情况下，各个记录的x字段所表示的是一个长度为3的数组：

In [28]:
arr[0]['x']

array([0., 0., 0.])

直接访问arr['x']可以得到一个二维数组：

In [29]:
arr['x']

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

> 注：dtype中还可以再次嵌套dtype

#### 为什么要使用结构化数组
跟pandas的DataFrame相比，Numpy的结构化数组是一个相对较低级的工具。它可以将单个内存块解释为带有任意复杂嵌套的表格型结构。由于数组中的每个元素在内存中都表示为`固定的字节数`，所以结构化数组能够提供非常`快速高效`的磁盘数据读写、网络传输等功能。
结构化数组另一个比较常见的用法：将数据文件写成定长记录字节流，这是C和C++代码中常见的数据序列化手段。只要知道文件的格式，就可以用numpy.fromfile将数据读入内存。